In [66]:
import json
from database.models import Message, Session, JoinedMember, Group
from datetime import datetime
from sqlalchemy import  select, func
from sqlalchemy import desc
from sqlalchemy.sql.functions import coalesce

def get_session():
    session = Session()
    try:
        yield session
    except Exception:
        session.rollback()
    finally:
        session.close()
session:Session = get_session().__next__()

In [2]:
query = session.query(Group, Message).outerjoin(
    Message, Group.id == Message.groupId
).group_by(Group.id).order_by(desc(Message.createdTime))

In [ ]:
query.all()

In [68]:
lstGrp = session.query(Group.id, Group.groupName, Group.createdTime).join(JoinedMember, Group.id == JoinedMember.groupId).distinct(JoinedMember.groupId).filter(JoinedMember.memberId == '3622BBFB-EE0A-47').subquery()

In [69]:
session.query(lstGrp).all()

2023-09-13 14:20:00,060 INFO sqlalchemy.engine.Engine BEGIN (implicit)


2023-09-13 14:20:00,069 INFO sqlalchemy.engine.Engine SELECT anon_1.[Id] AS [anon_1_Id], anon_1.[GroupName] AS [anon_1_GroupName], anon_1.[CreatedTime] AS [anon_1_CreatedTime] 
FROM (SELECT DISTINCT [Group].[Id] AS [Id], [Group].[GroupName] AS [GroupName], [Group].[CreatedTime] AS [CreatedTime] 
FROM [Group] JOIN [JoinedMember] ON [Group].[Id] = [JoinedMember].[GroupId] 
WHERE [JoinedMember].[MemberId] = ?) AS anon_1
2023-09-13 14:20:00,071 INFO sqlalchemy.engine.Engine [generated in 0.00435s] ('3622BBFB-EE0A-47',)


C:\Users\MinhThanh\AppData\Local\Temp\ipykernel_26752\4044168025.py:1: SADeprecationWarning: DISTINCT ON is currently supported only by the PostgreSQL dialect.  Use of DISTINCT ON for other backends is currently silently ignored, however this usage is deprecated, and will raise CompileError in a future release for all backends that do not support this syntax.
  session.query(lstGrp).all()


[('3B75B494-088E-48CB-81EE-DDF6AE090777', 'Nhóm số chín', datetime.datetime(2023, 7, 17, 14, 30, 27, 877000)),
 ('7D39C19E-47AE-46CF-8AC2-672B9FFE32E2', 'Nhóm số sáu', datetime.datetime(2023, 7, 16, 9, 19, 1, 850000)),
 ('9448AD90-AD01-4D4E-924C-E7E3D878AB63', '789ten', datetime.datetime(2023, 7, 16, 9, 10, 55, 923000)),
 ('C106161E-87FE-4F97-B4F0-2C40F2723F68', 'Nhóm số năm', datetime.datetime(2023, 7, 16, 9, 19, 1, 847000)),
 ('C9F57F41-6FDC-4929-8483-708717800999', 'Nhóm số bốn', datetime.datetime(2023, 7, 16, 9, 10, 55, 930000)),
 ('DAB5945C-F3CF-4EF4-BB53-84AD52C7F5ED', 'Nhóm số tám', datetime.datetime(2023, 7, 16, 9, 19, 1, 850000)),
 ('DCF9CCB8-DED0-4D12-BE1F-8467ADB76DFA', 'Nhóm số bảy', datetime.datetime(2023, 7, 16, 9, 19, 1, 850000)),
 ('EB9E82C9-245F-4177-A8E2-222637F1572F', 'Nhóm số ba', datetime.datetime(2023, 7, 16, 9, 10, 55, 930000)),
 ('FDA7D486-6EFB-4EFA-9650-C3879978C8B1', 'Nhóm số hai', datetime.datetime(2023, 7, 16, 9, 10, 55, 930000))]

In [70]:
subquery = session.query(
    func.rank().over(order_by=Message.createdTime.desc(), partition_by=Message.groupId).label('rnk'),
    Message.id, Message.content, Message.groupId, Message.senderId, Message.createdTime
).subquery()
subquery

<sqlalchemy.sql.selectable.Subquery at 0x28becfa9e50; anon_1>

In [73]:
# Query the top-ranked message from each group
query = session.query(lstGrp, subquery).outerjoin(
    subquery, lstGrp.c.id == subquery.c.groupId, full=True
).order_by(coalesce(subquery.c.createdTime, lstGrp.c.createdTime).desc()).filter(
    (subquery.c.rnk == 1)| (subquery.c.rnk == None)
).all()


In [78]:
# Execute the query and fetch all results
rows = query.all()

In [84]:
json.dump([list(row) for row in rows])

TypeError: dump() missing 1 required positional argument: 'fp'